In [1]:
import os
import pickle
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import Ollama

# Paths
MODEL = "mistral"  # Use the Mistral model
model = Ollama(model=MODEL)

text_chunks_folder = 'output/chunks/'
faiss_index_path = 'model_embeddings/faiss_index.index'
vector_db_path = 'model_embeddings/embeddings.pkl'

def load_text_chunks_from_folder(folder_path):
    text_chunks = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                text_chunks.append(file.read())
    return text_chunks

def embed_text_chunks(text_chunks, model):
    return model.encode(text_chunks)

def save_embeddings(embeddings, text_chunks, path):
    with open(path, 'wb') as file:
        pickle.dump((text_chunks, embeddings), file)

def load_embeddings(path):
    with open(path, 'rb') as file:
        return pickle.load(file)

def build_faiss_index(embeddings):
    dim = embeddings.shape[1]  # Dimension of embeddings
    index = faiss.IndexFlatL2(dim)  # L2 distance index
    index.add(embeddings)  # Add embeddings to index
    return index

from transformers import AutoTokenizer, AutoModelForCausalLM

def format_output(text):
    from langchain_community.llms import Ollama


def format_output(context, question):
    """
    Use Mistral model to generate formatted output.
    """
    # Define the template
    template = """
    Answer the question based on the context below. If you can't 
    answer the question, reply "I don't know".
    Only give me the answers based on the context below.
    Only answer the question asked. Do not provide additional information.
    Give a clear and concise answer.
    


    Context: {context}

    Question: {question}
    """

    # Format the template with context and question
    prompt_text = template.format(context=context, question=question)

    # Generate a response using the Mistral model
    response = model(prompt_text)
    # Return the formatted output
    return response


# Example usage





def search_faiss(query, index, model, k=5):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)  # Search for top-k similar embeddings
    return I[0]  # Returns the indices of the most similar chunks

def retrieve_and_format_results(query, index, text_chunks, model):
    indices = search_faiss(query, index, model)
    
    # Handle case where no indices are returned
    if not indices.size:
        return "No relevant information found."

    # Check for valid indices
    valid_indices = [i for i in indices if 0 <= i < len(text_chunks)]
    results = " ".join([text_chunks[i] for i in valid_indices])  # Concatenate retrieved chunks
    
    formatted_results = format_output(results ,query)
    return formatted_results

# Initialize models
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Load or create embeddings and FAISS index
if os.path.exists(vector_db_path):
    text_chunks, embeddings = load_embeddings(vector_db_path)
else:
    text_chunks = load_text_chunks_from_folder(text_chunks_folder)
    embeddings = embed_text_chunks(text_chunks, embedding_model)
    save_embeddings(embeddings, text_chunks, vector_db_path)

faiss_index = build_faiss_index(embeddings)

# Example usage


c:\Users\SURYA\anaconda3\envs\RAG\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\SURYA\anaconda3\envs\RAG\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


 I don't have the specific question from the context provided. Could you please provide the exact question for me to answer?


c:\Users\SURYA\anaconda3\envs\RAG\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:

query = "what are DNA made up of, explain in detail with help of flowchart" 
formatted_results = retrieve_and_format_results(query, faiss_index, text_chunks, embedding_model)
print("RAG :(\n")
print(formatted_results)


RAG :(

 DNA (Deoxyribonucleic Acid) is a long, twisted molecule composed primarily of two strands that are complementary to each other. Each side rail of the DNA ladder is made up of alternating sugar and phosphate groups. The sugars are deoxyribose sugars, hence the name Deoxyribonucleic Acid.

   Flowchart:
   1. Sugar: Deoxyribose (2-deoxypentose)
       - A 5-carbon sugar with a hydroxyl group at the second carbon and a hydrogen atom at the third carbon, instead of the usual ketone group that would be found in other pentose sugars.
   2. Phosphate:
       - A negatively charged molecule consisting of one phosphorus atom bonded to three oxygen atoms (a trioxophosphate).
   3. Bases: There are four types of nitrogenous bases in DNA: adenine (A), thymine (T), guanine (G), and cytosine (C)
       - These bases pair up specifically during the formation of the double helix structure, with A pairing with T, and C pairing with G.
   4. Phosphodiester bond: The nucleotides are linked toget